# Demand Price Elasticity

Major business and marketers are always keen to know the art behind setting an optimal price point of their product. we often think what could be the factors that a company need to consider while setting a price of product and how we can measure the consumer demand impact towards the price settled? Do the product price changes affect my consumer demand? I have unfolded facts about relationship between sales demand and price fluctuation

I have divided this whole analysis into three section.
1. Data Cleaning & Pre-processing - [https://www.kaggle.com/code/yugagrawal95/price-elasticity-data-cleaning-part-1]
2. Exploratory Data Analysis - [https://www.kaggle.com/code/yugagrawal95/price-elasticity-eda-part-2]
3. Price Elasticity Model - [https://www.kaggle.com/code/yugagrawal95/price-elasticity-model-part-3]

In case, You are just intrested in EDA or direct modelling related stuff, feel free to use other kernals

### Load all the required packages that will be leveraged in the below code

In [1]:
#load all the libraries...
import pandas as pd
import numpy as np
import re
from IPython.display import display, HTML, Image

#imported warning to remove extra warning log from code
import warnings
warnings.filterwarnings('ignore')

### Load the data set. Used Electronic product data and I have used Kaggle data set. Link here 

In [4]:
# df = pd.read_csv('DatafinitiElectronicsProductsPricingData.csv')
import os
os.chdir("C:\\Users\\alwar\\Documents\\Datascience\\Python\\price_elasticity_demand\\dataset")
df = pd.read_csv('DatafinitiElectronicsProductsPricingData.csv')

In [5]:
#Setting the display option to max 500 as our dataset has more than 20 column and it will be hard to see all those with default value
pd.options.display.width = 0
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',500)

In [6]:
#Some General Things before starting..
print("Total Numbers of Rows in data set",df.shape[0])
print("Total Numbers of columns in data set",df.shape[1])
print(f"Costliest Product of data - ${df['prices.amountMax'].max()} USD")
print(f"Cheapest Product of data - ${df['prices.amountMax'].min()} USD")

df.head(2)

Total Numbers of Rows in data set 14592
Total Numbers of columns in data set 31
Costliest Product of data - $26871.0 USD
Cheapest Product of data - $1.0 USD


,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,prices.sourceURLs,asins,brand,categories,dateAdded,dateUpdated,ean,imageURLs,keys,manufacturer,manufacturerNumber,name,primaryCategories,sourceURLs,upc,weight,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",False,Bestbuy.com,NaN,http://www.bestbuy.com/site/sanus-tv-wall-moun...,B00C78VIUE,Sanus,"Audio & Video Accessories,TV Mounts,TV Accesso...",2015-04-13T12:00:51Z,2018-05-12T18:59:48Z,NaN,https://images-na.ssl-images-amazon.com/images...,sanusvlf410b110inchsuperslimfullmotionmountfor...,NaN,VLF410B1,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.93796E+11,32.8 pounds,NaN,NaN,NaN,NaN,NaN
1,AVpgMuGwLJeJML43KY_c,69.00,64.99,In Stock,New,USD,2017-12-14T06:00:00Z,True,Walmart.com,Expedited,https://www.walmart.com/ip/BOYTONE-BT210FB-BLA...,"B018K251JE,B00VILQKQ8",Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,NaN,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds,NaN,NaN,NaN,NaN,NaN


In [7]:
# Check the column level Null %
# Check data Type of each of the column
temp_df = pd.DataFrame((df.isnull().sum()/df.shape[0])*100,columns=['Null value%'])
temp_df['data_type'] = df.dtypes
temp_df.T

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,prices.sourceURLs,asins,brand,categories,dateAdded,dateUpdated,ean,imageURLs,keys,manufacturer,manufacturerNumber,name,primaryCategories,sourceURLs,upc,weight,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
Null value%,0.0,0.0,0.0,13.19216,9.614857,0.0,0.0,0.0,5.29057,39.740954,0.0,0.0,0.0,0.0,0.0,0.0,71.162281,0.0,0.0,47.793311,0.0,0.0,0.0,0.0,0.0,0.0,99.73273,99.876645,99.958882,99.917763,99.958882
data_type,object,float64,float64,object,object,object,object,bool,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,object,object


In [8]:
# Price Availbility has been categorized in 3 terms -
# - Present or in stock - Yes, Not present - No, Special order - Special

df['prices.availability'] = np.where(df['prices.availability'].str.contains('Yes|TRUE|In Stock|yes|available', flags=re.IGNORECASE), "Yes",
                 np.where(df['prices.availability'].str.contains('No|sold|FALSE|Retired|Discontinued', flags=re.IGNORECASE), "No",
                 np.where(df['prices.availability'].str.contains('Special Order|More on the Way|More Coming Soon', flags=re.IGNORECASE), "Special",np.NaN
                 )))

In [9]:
# We can use this column and extract the condition of the proudct
# It is either New, Used or Refurbished 
# df['prices.condition'].unique() 

df.loc[df['prices.condition'].str.contains('new',flags=re.IGNORECASE) == True,"prices.condition"] = 'New'
df.loc[df['prices.condition'].str.contains('refurbished',flags=re.IGNORECASE) == True,"prices.condition"] = 'Refurbished'
df.loc[df['prices.condition'].str.contains('pre-owned|used',flags=re.IGNORECASE) == True,"prices.condition"] = 'Used'

### Impression count calculation

Here we don't have impression count or quantity sold but our end goal is to check impact of price in quantity. so we need create this feature.
dateseen is number of times product has been seen in that day/week/month etc. Hence lets assume every seen is 1 quantity sold.

Steps - Split the dateseen and count the length of dates that serves as impression

Also, convert date seen into date column so we can utilize its date component

In [10]:
Impression_count=[]
for i in df['prices.dateSeen']:
    time_= i.split(",")
    Impression_count.append(len(time_))

df['Impression'] = Impression_count

df['prices.dateSeen'] = df['prices.dateSeen'].apply(lambda x: x.split(",")[0])
df['prices.dateSeen'] = pd.to_datetime(df['prices.dateSeen'])

In [11]:
# we noticed some anamolies while we checked isSale column, 
# Analogy is if isSale is true then Actual Price != Discount Price because there would be some discount in sale
#if isSale is False then Actual Price == Discount Price because there won't be any discount in sale


df.loc[((df['prices.amountMax'] != df['prices.amountMin']) & (df['prices.isSale'] == False)),"prices.isSale"] = True
df.loc[((df['prices.amountMax'] == df['prices.amountMin']) & (df['prices.isSale'] == True)),"prices.isSale"] = False

### Merchant Imputation

From SourceURLs we can extract merchant information

In [12]:
# df[df['prices.merchant'].isnull()]
def Source_to_Merchant(x):
   try:
    return x.split("www.")[1].split("/")[0]
   except:
    if len(x)>0:
        return x.split("//")[1].split(".com")[0]
    return x

df['prices.sourceURLs'] = df['prices.sourceURLs'].apply(lambda x: Source_to_Merchant(x))
df.loc[df['prices.merchant'].isnull(),'prices.merchant'] = df['prices.sourceURLs']

In [13]:
#Setting merchant to a finite list as most of the chunk is covered by these big vendors only

df['prices.merchant'] = np.where(df['prices.merchant'].str.contains('bhphotovideo', flags=re.IGNORECASE), "bhphotovideo.com",
                 np.where(df['prices.merchant'].str.contains('eBay|e bay', flags=re.IGNORECASE), "ebay.com",
                 np.where(df['prices.merchant'].str.contains('Amazon',flags=re.IGNORECASE), "Amazon.com", 
                 np.where(df['prices.merchant'].str.contains('Bestbuy',flags=re.IGNORECASE), "Bestbuy.com",
                 np.where(df['prices.merchant'].str.contains('Homedepot',flags=re.IGNORECASE), "homedepot.com",
                 np.where(df['prices.merchant'].str.contains('newegg',flags=re.IGNORECASE), "newegg.com",
                 np.where(df['prices.merchant'].str.contains('kmart',flags=re.IGNORECASE), "kmart.com",
                 np.where(df['prices.merchant'].str.contains('frys',flags=re.IGNORECASE), "frys.com",
                 np.where(df['prices.merchant'].str.contains('cdw',flags=re.IGNORECASE), "cdw.com",
                 np.where(df['prices.merchant'].str.contains('target',flags=re.IGNORECASE), "target.com",
                 np.where(df['prices.merchant'].str.contains('overstock',flags=re.IGNORECASE), "overstock.com",
                 np.where(df['prices.merchant'].str.contains('barcodable',flags=re.IGNORECASE), "barcodable.com",
                 np.where(df['prices.merchant'].str.contains('kohls',flags=re.IGNORECASE), "kohls.com",
                 np.where(df['prices.merchant'].str.contains('sears',flags=re.IGNORECASE), "sears.com",
                 np.where(df['prices.merchant'].str.contains('Wal-mart|Walmart',flags=re.IGNORECASE), "Walmart.com","Other")))))))))))))))

In [14]:
#convert date with date time features, currently it is in string type

df['dateAdded'] = pd.to_datetime(df['dateAdded'])
df['dateUpdated'] = pd.to_datetime(df['dateUpdated'])

In [15]:
#Engineer Country Variable based on the currency

df['Country']= np.where(df['prices.currency'] == 'USD','USA',
np.where(df['prices.currency'] == 'CAD',"Canada",
np.where(df['prices.currency'] == 'SGD',"Singapore",
np.where(df['prices.currency'] == 'EUR',"EUROPE",
np.where(df['prices.currency'] == 'GBP',"UK","Other"
)))))

In [17]:
#Convert weight into KG, currently weights are in various format so to bring in same scale
# KG = Pounds*0.453592 = lbs*0.453592 = oz*0.453592

weight_list = []
for x in df['weight'].to_list():  
    if ((x.find('lb') !=-1) or (x.find('lbs') !=-1) ):
        try:
            weight_list.append(float(x.split("lbs")[0].strip())*0.453592)
        except:
            try:
                weight_list.append(float(x.split("lb")[0].strip())*0.453592)
            except:
                weight_list.append(np.NaN)
    elif 'pounds' in x:
        weight_list.append(float(x.split("pounds")[0].strip())*0.453592)
    elif 'ounces' in x:
        weight_list.append(float(x.split("ounces")[0].strip())*0.453592)
    elif 'oz' in x:
        weight_list.append(float(x.split("oz")[0].strip())*0.453592)
    elif (('Kg' in x) or ('kg' in x)) :
        try:
            weight_list.append(float(x.split("Kg")[0].strip()))
        except:
            weight_list.append(float(x.split("kg")[0].strip()))
    elif 'g' in x:
        try:
            weight_list.append(float(x.split("g")[0].strip())/1000)
        except:
            weight_list.append(np.NaN)  
    else:
        weight_list.append(np.NaN)

df['weight_in_kg'] = weight_list

### Importing Data set with category name and cluster

Category Name and cluster has been defined using TF-IDF vectorizer functionality. If you need detail for that, reach out to me

In [20]:
# category_name = pd.read_csv('DatafinitiElectronicsProductsPricingData_with_category.csv')
category_name = pd.read_csv('DatafinitiElectronicsProductsPricingData_with_category.csv')

In [21]:
df['Cluster'] = category_name['Cluster']
df['Category_name'] = category_name['Category_name']

In [25]:
#Renaming the column name

df.rename(columns={'prices.amountMax':'Actual_price',
'prices.amountMin':'Discount_price',
'prices.availability':'Product_availability',
'prices.condition':'Condition',
'prices.currency':'Currency',
'prices.isSale':'isSale',
'prices.merchant':'merchant',
'prices.dateSeen':'Date',
'categories':'Description'
},inplace=True)


In [23]:
#Select important features that we are using ahead..

Sub_df = df[['id','Actual_price','Discount_price','Product_availability','Condition','Currency','Country',
'Date','isSale','merchant','brand','Description','Category_name','primaryCategories','name','weight_in_kg','Impression']]

In [26]:
#Generate date level feature such as day, month, week, month name etc.

Sub_df['Day'] = Sub_df['Date'].dt.day
Sub_df['Month'] = Sub_df['Date'].dt.month
Sub_df['Year'] = Sub_df['Date'].dt.year
Sub_df['Week'] = Sub_df['Date'].dt.dayofweek
Sub_df['Month_Name'] = Sub_df['Date'].dt.strftime('%B')
Sub_df['Day_Name'] = Sub_df['Date'].dt.strftime('%A')
Sub_df['Formatted_date'] = pd.to_datetime(Sub_df['Date'].dt.date)

In [27]:
#Calculate the discount percentage based on the Actual & Discounting price
Sub_df['Disc_Per'] = ((Sub_df['Actual_price'] - Sub_df['Discount_price'])/Sub_df['Actual_price'])*100

In [28]:
# We can generate Total sales as well based on Impression and Discount price
Sub_df['Total_sale'] = Sub_df['Impression'] * Sub_df['Discount_price']

In [29]:
#Final check in data types and Null values
temp_df = pd.DataFrame((Sub_df.isnull().sum()/Sub_df.shape[0])*100,columns=['Null value%'])
temp_df['data_type'] = Sub_df.dtypes
temp_df.T

,id,Actual_price,Discount_price,Product_availability,Condition,Currency,Country,Date,isSale,merchant,brand,Description,Category_name,primaryCategories,name,weight_in_kg,Impression,Day,Month,Year,Week,Month_Name,Day_Name,Formatted_date,Disc_Per,Total_sale
Null value%,0.0,0.0,0.0,0.0,9.614857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.452303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
data_type,object,float64,float64,object,object,object,object,"datetime64[ns, UTC]",bool,object,object,object,object,object,object,float64,int64,int32,int32,int32,int32,object,object,datetime64[ns],float64,float64


In [30]:
#Remove duplicates 
Sub_df = Sub_df[~Sub_df.duplicated()]

In [31]:
#Drop date column as we have extracted all the date features
Sub_df.drop(columns=['Date'],inplace=True)

In [32]:
# Sub_df.to_excel('Cleaned_dataset.xlsx',index=False)
Sub_df.to_csv('./DatafinitiElectronicsProductsPricingData_Cleaned_dataset.csv',index=False)